In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import geopandas as gpd
import shapely
from shapely import Point, Polygon, LineString
import folium

# Taller 1

## Adjunto a este script encontrará un conjunto de datos (.csv) el cual contiene valores de concentración de metano (en ppb) promedio desde el año 2019 al 2024 en algunas zonas del territorio colombiano. A partir de esta información:

### 1. Muestre los datos en el explorador de geopandas

In [ ]:
CH4 = pd.read_csv("CH4_mean_2019-2024.csv")

In [ ]:
CH4_gdf = gpd.GeoDataFrame(CH4, geometry=gpd.points_from_xy(CH4.x, CH4.y),crs="EPSG:4326")

In [ ]:
CH4_gdf.head(4)

In [ ]:
CH4_gdf.explore(column="CH4ppb",cmap="Set1", style_kwds=dict(marker_type="circle_marker"))

### 2. Estime y proyecte el centro medio y centro medio ponderado (variable CH4)

In [ ]:
MeanX = CH4_gdf.x.mean()
MeanY = CH4_gdf.y.mean()

In [ ]:
MeanX

In [ ]:
MeanY

In [ ]:
MeanWx = (CH4_gdf.x * CH4_gdf.CH4ppb).sum() / CH4_gdf.CH4ppb.sum()

In [ ]:
MeanWy = (CH4_gdf.y * CH4_gdf.CH4ppb).sum() / CH4_gdf.CH4ppb.sum()

In [ ]:
MeanWx

In [ ]:
MeanWy

In [ ]:
Mean = pd.DataFrame({"x":[MeanX], "y":[MeanY]})
MeanW = pd.DataFrame({"x":[MeanWx], "y":[MeanWy]})

In [ ]:
Mean = gpd.GeoDataFrame(Mean, geometry=gpd.points_from_xy(Mean.x, Mean.y),crs="EPSG:4326")

In [ ]:
MeanW = gpd.GeoDataFrame(MeanW, geometry=gpd.points_from_xy(MeanW.x, MeanW.y),crs="EPSG:4326")

In [ ]:
m= CH4_gdf.explore(column="CH4ppb",cmap="Set1", style_kwds=dict(marker_type="circle_marker"))
m= Mean.explore(m=m,color="red",marker_kwds=dict(radius=10, fill=True))
MeanW.explore(m=m,color="green",marker_kwds=dict(radius=10, fill=True))

### 3. Estime y proyecte la mediana con base en la longitud, latitud y las concentraciones de CH4

In [ ]:
MedianX = CH4_gdf.x.median()
MedianY = CH4_gdf.y.median()

In [ ]:
MedianX

In [ ]:
MedianY

In [ ]:
Median = pd.DataFrame({"x":[MedianX], "y":[MedianY]})

In [ ]:
Median = gpd.GeoDataFrame(Median, geometry=gpd.points_from_xy(Median.x, Median.y),crs="EPSG:4326")

In [ ]:
CH4_gdf.describe()

In [ ]:
m= CH4_gdf.explore(column="CH4ppb",cmap="Set1", style_kwds=dict(marker_type="circle_marker"))
m= Mean.explore(m=m,color="red",marker_kwds=dict(radius=10, fill=True))
m= MeanW.explore(m=m,color="green",marker_kwds=dict(radius=10, fill=True))
Median.explore(m=m,color="blue",marker_kwds=dict(radius=10, fill=True))

### 4. Estime y analice brevemente parametros descriptivos como el promedio, la mediana, el mínimo y el máximo. Tenga en cuenta que al llegar a este punto ya conoce el promedio y la mediana de las concentraciones de CH4. Que relación tienen con el posicionamiento espacial en que marcan dichos resultados. Respecto a la moda, proyecto un histograma de las concentraciones de CH4 (las cuales están en ppb).

In [ ]:
CH4_gdf[CH4_gdf.CH4ppb == CH4_gdf.CH4ppb.max()]

In [ ]:
CH4_gdf[CH4_gdf.CH4ppb == CH4_gdf.CH4ppb.min()]

In [ ]:
Max = pd.DataFrame({"x":[-74.845342], "y":[4.30502]})
Max = gpd.GeoDataFrame(Max, geometry=gpd.points_from_xy(Max.x, Max.y),crs="EPSG:4326")

In [ ]:
Min = pd.DataFrame({"x":[-73.745337], "y":[2.595012]})
Min = gpd.GeoDataFrame(Min, geometry=gpd.points_from_xy(Min.x, Min.y),crs="EPSG:4326")

In [ ]:
m= Mean.explore(m=m,color="red",marker_kwds=dict(radius=10, fill=True))
m= MeanW.explore(m=m,color="green",marker_kwds=dict(radius=10, fill=True))
m = Median.explore(m=m,color="blue",marker_kwds=dict(radius=10, fill=True))
m = Max.explore(m=m,color="yellow",marker_kwds=dict(radius=10, fill=True))
Min.explore(m=m,color="orange",marker_kwds=dict(radius=10, fill=True))

In [ ]:
plt.hist(CH4_gdf.CH4ppb, bins=25)
plt.xlabel("$CH_4$ (ppb)")

### 5. Estime y proyecto la desviación estándar de los datos

In [ ]:
promX = CH4_gdf.x.mean()
promY = CH4_gdf.y.mean()
Promedio = gpd.GeoSeries([Point(promX,promY)])

medX = CH4_gdf.x.median()
medY = CH4_gdf.y.median()
Mediana = gpd.GeoSeries([Point(medX,medY)])

w= 1
x_i = (CH4_gdf.x * w)-promX
y_i = (CH4_gdf.y * w)-promY
x_i2 = (x_i**2).sum()
y_i2 = (y_i**2).sum()
xi2_yi2 = x_i2-y_i2
xiyi = (x_i*y_i).sum()
xiyi2 = xiyi**2
theta = np.arctan((xi2_yi2 + np.sqrt(np.abs(xi2_yi2) + 4*xiyi2))/(2*xiyi2))

n = len(CH4_gdf)
ysen = y_i* (np.sin(theta))
ycos = y_i*np.cos(theta)
xsen = x_i*np.sin(theta)
xcos = x_i*np.cos(theta)
sigmax = np.sqrt( (1/n) * ((ysen+xcos)**2).sum() )
sigmay = np.sqrt( (1/n) * ((ycos+xsen)**2).sum() )

In [ ]:
theta_degree = np.degrees(theta)
theta_degree

In [ ]:
ellipse = ((promX, promY),(sigmax, sigmay),theta_degree)

In [ ]:
print(ellipse[1][0], ellipse[1][1])

In [ ]:
ell  = shapely.affinity.scale(Point(ellipse[0]).buffer(1), ellipse[1][0], ellipse[1][1]) #tomamos sigmax y sigmay para crear elipse
ell

In [ ]:
ellr = shapely.affinity.rotate(ell,ellipse[2]) #rotamos la elipse
ellr

In [ ]:
ellr = gpd.GeoSeries([ellr])

In [ ]:
m = CH4_gdf.explore(column="CH4ppb", legend=False, color="black", marker_kwds=dict(radius=1, fill=True))
ellr.explore(m=m, color="yellow")